## Algorithmus von Floyd and Warshall

#### Kürzeste Wege

```pseudocode
procedure [array] FloydWarshall(D, P)
  for k in 1 to n do
    for i in 1 to n do
      for j in 1 to n do
        if D[i][j] > D[i][k] + D[k][j] then
          D[i][j] = D[i][k] + D[k][j]
          P[i][j] = P[k][j]
  return P
```
[Simulation](https://www.cs.usfca.edu/~galles/visualization/Floyd.html)

[Beispiele](https://compprog.wordpress.com/2007/11/15/all-sources-shortest-path-the-floyd-warshall-algorithm/)

In [1]:
D = [
    [0,10,0,5,0]
    [10,0,5,5,10]
    [0,5,0,0,20]
    [5,5,0,0,20]
    [0,10,0,20,0]
]

A = reshape(D, 5, 5)

5x5 Array{Int64,2}:
  0  10   0   5   0
 10   0   5   5  10
  0   5   0   0   0
  5   5   0   0  20
  0  10  20  20   0

In [3]:

using Graphs


print(D)
g = simple_adjlist(A, is_directed=false)
plot(g)


[0,10,0,5,0,10,0,5,5,10,0,5,0,0,20,5,5,0,0,20,0,10,0,20,0]

LoadError: `to_dot` has no method matching to_dot(::GenericAdjacencyList{Int64,UnitRange{Int64},Array{Array{Int64,1},1}}, ::Pipe)
while loading In[3], in expression starting on line 7